In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xlrd

sns.set(rc={"figure.figsize": (12, 10)})
pd.set_option("display.max_rows", 50000)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
stocks = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Stocks_DS_tab_delimited.dat", sep="\t", na_values=' ', parse_dates=["datadate"])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,6,11,14,15,16,17,18,19,20,21,38,42,43,44,45,50,52,53,60,64,65,70,71,74,75) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
fundamentals = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fundamentals_Full.csv", na_values=' ')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,25,29,916,917,919,920,926,935,937,940,941,942,943,945,955,956,959,962,965,970,978,981,984,989,992,993,997,998,1008,1009,1014,1015,1016,1017,1018,1019,1020,1021,1024,1025,1028,1029,1031,1033,1034,1035,1036,1040,1041,1042,1052,1053,1054,1055,1056,1057,1058,1059,1066,1071,1072,1074,1075,1076,1077,1081,1084,1091,1094,1096,1098,1101,1102,1103,1105,1106,1107,1108,1116,1117,1119,1120,1121,1122,1123,1124,1125,1735,1736,1737,1742,1756,1757,1762) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
fund_security = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fund_Settlement_Security.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,15,25,29,916,917,919,920,926,935,936,937,940,941,942,943,945,950,951,952,953,955,956,959,962,965,970,974,978,984,989,992,993,996,997,998,1008,1009,1014,1015,1016,1017,1018,1019,1020,1021,1024,1025,1028,1029,1030,1031,1033,1034,1035,1036,1038,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1052,1053,1054,1055,1056,1057,1058,1059,1066,1071,1072,1074,1075,1076,1077,1081,1082,1083,1084,1091,1094,1096,1098,1101,1102,1103,1105,1106,1107,1108,1113,1114,1115,1116,1117,1119,1120,1121,1122,1123,1124,1125,1735,1736,1737,1742,1756,1757,1762,1766) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def excel_to_dictionary(sheetname, filename="/content/drive/MyDrive/Fiverr/PROJECT_MIMI/DataDictionary.xlsx"):
    abbreviation_dict = dict()
    
    abbreviation = xlrd.open_workbook(os.path.abspath(filename))
    abbreviation_sheet = abbreviation.sheet_by_name(sheetname)
    
    for i in range(1, abbreviation_sheet.nrows):
        abb = abbreviation_sheet.cell(i, 0).value
        abb_definition = abbreviation_sheet.cell(i, 1).value
        abbreviation_dict[abb] = abb_definition
        
    return abbreviation_dict

In [7]:
stocks_dict = excel_to_dictionary(sheetname="Stocks")
fundamentals_dict = excel_to_dictionary(sheetname="Fundamentals")

In [8]:
def rename_columns(df, df_dict):
  df.rename(columns=df_dict, inplace=True)
  df.columns = [column.lower().replace(" ", "_") for column in df.columns]
  return df

In [9]:
stocks = rename_columns(stocks, stocks_dict)
fundamentals = rename_columns(fundamentals, fundamentals_dict)

In [10]:
initial_stocks = stocks["global_company_key"].nunique()
initial_stocks_columns = len(stocks.columns)
print("The stocks data file has {} companies while the fundamentals file has {} companies".format(initial_stocks, fundamentals["global_company_key"].nunique()))

The stocks data file has 4632 companies while the fundamentals file has 10555 companies


In [11]:
stocks["year"] = stocks["data_date_-_dividends"].apply(lambda x: x.strftime("%Y"))

In [12]:
shared_stock_companies = list(set(sorted(fundamentals["global_company_key"].unique())) & set(sorted(stocks["global_company_key"].unique())))

In [13]:
stocks = stocks[stocks["global_company_key"].isin(shared_stock_companies)]
fundamentals = fundamentals[fundamentals["global_company_key"].isin(shared_stock_companies)]

In [14]:
shared_stocks_columns = list(set(sorted(fundamentals.columns)) & set(sorted(stocks.columns)))

In [15]:
print("Stocks has dropped from {} to {} companies.\n\nStocks columns have dropped from {} to {}".format(
                                                                                                  initial_stocks, stocks["global_company_key"].nunique(), 
                                                                                                  initial_stocks_columns, stocks.shape[1]))

Stocks has dropped from 4632 to 2462 companies.

Stocks columns have dropped from 76 to 77


In [16]:
for column in stocks[shared_stocks_columns].columns:
  stocks.loc[stocks[column].isnull(), column] = "not_recorded"

In [17]:
stocks.shape

(2447018, 77)

In [18]:
stocks = stocks.groupby(["global_company_key", "year"])[['price_-_close_-_daily', 'price_-_high_-_daily', 'price_-_low_-_daily', 'price_-_open_-_daily', 'price_status_code_-_daily']].agg("sum").reset_index()

In [19]:
stocks.shape

(10616, 7)

In [20]:
stocks = stocks.groupby(["global_company_key"])[['price_-_close_-_daily', 'price_-_high_-_daily', 'price_-_low_-_daily', 'price_-_open_-_daily', 'price_status_code_-_daily']].agg("mean").reset_index()

In [21]:
stocks.shape

(2462, 6)

In [22]:
fund_stocks = pd.merge(fund_security, stocks, on="global_company_key")
fund_stocks.shape

In [23]:
fund_stocks[['price_-_close_-_daily', 'price_-_high_-_daily', 'price_-_low_-_daily', 'price_-_open_-_daily', 'price_status_code_-_daily', "global_company_key", "dividend_rate_-_monthly", 'dividends_per_share_-_ex_date_-_monthly', 'dividends_per_share_-_pay_date_-_monthly', 'common_stock_float_shares_-_canada', 'iso_currency_code_-_monthly', "settlementamount", "sued", "global_company_key", "ticker_symbol"]].head()

,price_-_close_-_daily,price_-_high_-_daily,price_-_low_-_daily,price_-_open_-_daily,price_status_code_-_daily,global_company_key,dividend_rate_-_monthly,dividends_per_share_-_ex_date_-_monthly,dividends_per_share_-_pay_date_-_monthly,common_stock_float_shares_-_canada,iso_currency_code_-_monthly,settlementamount,sued,global_company_key,ticker_symbol
0,6949.33256,7025.21376,6866.63048,6798.90896,833.2,1076,0.808,0.068,0.0636,0.0,USD,NaN,no,1076,AAN
1,6949.33256,7025.21376,6866.63048,6798.90896,833.2,1076,0.808,0.068,0.0636,0.0,USD,NaN,no,1076,AAN
2,6949.33256,7025.21376,6866.63048,6798.90896,833.2,1076,0.808,0.068,0.0636,0.0,USD,NaN,no,1076,AAN
3,6949.33256,7025.21376,6866.63048,6798.90896,833.2,1076,0.808,0.068,0.0636,0.0,USD,NaN,no,1076,AAN
4,6949.33256,7025.21376,6866.63048,6798.90896,833.2,1076,0.808,0.068,0.0636,0.0,USD,NaN,no,1076,AAN


In [24]:
# for company in [11300, 146156, 11017]:
#   subset = stocks_[stocks_["global_company_key"] == company]

#   sns.distplot(subset["cash_dividends_-_daily"].dropna(), hist=False, kde=True, label=company)

# plt.xlabel("Daily Cash Dividends"); plt.ylabel("Density");
# plt.title("Density Plots for Daily Cash Dividends")
# plt.legend()

# # stocks_[['cash_dividends_-_daily', "global_company_key"]].head(10)

In [25]:
fund_stocks.to_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fund_Settlement_Security_Stocks.csv", index=False)